In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from matplotlib import cm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA

### PART 4: Running Neural Network on Dimentionality Reduction Algorithms

In [7]:
np.random.seed(0)
breast_cancer = pd.read_csv('./breast_cancer.csv')
bc_X = breast_cancer.drop('class',1).copy().values
bc_Y = breast_cancer['class'].copy().values

In [9]:
breast_cancer

,RealValuedInputFeature_1,RealValuedInputFeature_2,RealValuedInputFeature_3,RealValuedInputFeature_4,RealValuedInputFeature_5,RealValuedInputFeature_6,RealValuedInputFeature_7,RealValuedInputFeature_8,RealValuedInputFeature_9,RealValuedInputFeature_10,...,RealValuedInputFeature_22,RealValuedInputFeature_23,RealValuedInputFeature_24,RealValuedInputFeature_25,RealValuedInputFeature_26,RealValuedInputFeature_27,RealValuedInputFeature_28,RealValuedInputFeature_29,RealValuedInputFeature_30,class
0,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.300100,0.147100,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.71190,0.26540,0.4601,0.11890,M
1,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.086900,0.070170,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.24160,0.18600,0.2750,0.08902,M
2,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.197400,0.127900,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.45040,0.24300,0.3613,0.08758,M
3,11.420,20.38,77.58,386.1,0.14250,0.28390,0.241400,0.105200,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.68690,0.25750,0.6638,0.17300,M
4,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.198000,0.104300,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.40000,0.16250,0.2364,0.07678,M
5,12.450,15.70,82.57,477.1,0.12780,0.17000,0.157800,0.080890,0.2087,0.07613,...,23.75,103.40,741.6,0.17910,0.52490,0.53550,0.17410,0.3985,0.12440,M
6,18.250,19.98,119.60,1040.0,0.09463,0.10900,0.112700,0.074000,0.1794,0.05742,...,27.66,153.20,1606.0,0.14420,0.25760,0.37840,0.19320,0.3063,0.08368,M
7,13.710,20.83,90.20,577.9,0.11890,0.16450,0.093660,0.059850,0.2196,0.07451,...,28.14,110.60,897.0,0.16540,0.36820,0.26780,0.15560,0.3196,0.11510,M
8,13.000,21.82,87.50,519.8,0.12730,0.19320,0.185900,0.093530,0.2350,0.07389,...,30.73,106.20,739.3,0.17030,0.54010,0.53900,0.20600,0.4378,0.10720,M
9,12.460,24.04,83.97,475.9,0.11860,0.23960,0.227300,0.085430,0.2030,0.08243,...,40.68,97.65,711.4,0.18530,1.05800,1.10500,0.22100,0.4366,0.20750,M


In [10]:
bc_X= StandardScaler().fit_transform(bc_X)

### ICA on Breast Cancer

In [11]:
clusters =  [2,5,10,15,20,25,30,35,40]
dims = [2,5,10,15,20,25,30,35,40,45,50,55,60]

In [12]:
out = './ICA/'
ica = FastICA(random_state=5)
kurt = {}
for dim in dims:
    ica.set_params(n_components=dim)
    tmp = ica.fit_transform(bc_X)
    tmp = pd.DataFrame(tmp)
    tmp = tmp.kurt(axis=0)
    kurt[dim] = tmp.abs().mean()

kurt = pd.Series(kurt) 
kurt.to_csv(out+'breast_cancer.csv')

/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/decomposition/fastica_.py:301: UserWarning: n_components is too large: it will be set to 30
  warnings.warn('n_components is too large: it will be set to %s' % n_components)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/decomposition/fastica_.py:301: UserWarning: n_components is too large: it will be set to 30
  warnings.warn('n_components is too large: it will be set to %s' % n_components)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/decomposition/fastica_.py:301: UserWarning: n_components is too large: it will be set to 30
  warnings.warn('n_components is too large: it will be set to %s' % n_components)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/decomposition/fastica_.py:301: UserWarning: n_components is too large: it will be set to 30
  warnings.warn('n_components is too large: it will be set to %s' % n_components)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/decomposi

In [22]:
nn_arch= [(50,50),(50,),(25,),(25,25),(100,25,100)]
nn_reg = [10**-x for x in range(1,5)]
clusters =  [2,5,10,15,20,25,30,35,40]
dims = [2,5,10,15,20]
grid ={'ica__n_components':dims,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
ica = FastICA(random_state=5, max_iter=1000)       
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
pipe = Pipeline([('ica',ica),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10,cv=5)

gs.fit(bc_X,bc_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'Breast Cancer dim red.csv')

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.6260869565217392, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.6260869565217392, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.6283185840707964, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.6283185840707964, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.6283185840707964, total=

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s remaining:    0.0s



[CV]  NN__alpha=0.1, ica__n_components=10, NN__hidden_layer_sizes=(50, 50), score=0.6260869565217392, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=10, NN__hidden_layer_sizes=(50, 50) 


/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/decomposition/fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


[CV]  NN__alpha=0.1, ica__n_components=10, NN__hidden_layer_sizes=(50, 50), score=0.6173913043478261, total=   0.3s
[CV] NN__alpha=0.1, ica__n_components=10, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.1, ica__n_components=10, NN__hidden_layer_sizes=(50, 50), score=0.6460176991150443, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=10, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.1, ica__n_components=10, NN__hidden_layer_sizes=(50, 50), score=0.6283185840707964, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=10, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.1, ica__n_components=10, NN__hidden_layer_sizes=(50, 50), score=0.6283185840707964, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=15, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.1, ica__n_components=15, NN__hidden_layer_sizes=(50, 50), score=0.6347826086956522, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=15, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.1, ica__n_components=15, NN

...

/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/decomposition/fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


[CV]  NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(25, 25), score=0.6260869565217392, total=   0.4s
[CV] NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(25, 25), score=0.6283185840707964, total=   0.0s
[CV] NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(25, 25), score=0.6283185840707964, total=   0.0s
[CV] NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(25, 25), score=0.6283185840707964, total=   0.0s
[CV] NN__alpha=0.0001, ica__n_components=15, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, ica__n_components=15, NN__hidden_layer_sizes=(25, 25), score=0.3739130434782609, total=   0.0s
[CV] NN__alpha=0.0001, ica__n_components=15, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha

/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/decomposition/fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


[CV]  NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(100, 25, 100), score=0.6260869565217392, total=   0.4s
[CV] NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(100, 25, 100), score=0.6283185840707964, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(100, 25, 100), score=0.6283185840707964, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, ica__n_components=10, NN__hidden_layer_sizes=(100, 25, 100), score=0.6460176991150443, total=   0.0s
[CV] NN__alpha=0.0001, ica__n_components=15, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, ica__n_components=15, NN__hidden_layer_sizes=(100, 25, 100), score=0.6434782608695652, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_component

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   29.4s finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

### ICA on Wine Quality Dataset

In [31]:
wine_quality = pd.read_csv('./winequality-white-v2.csv')
wq_X = wine_quality.drop('quality',1).copy().values
wq_Y = wine_quality['quality'].copy().values

In [45]:
wine_quality

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.270,0.36,20.70,0.045,45.0,170.0,1.00100,3.00,0.45,8.800000,Normal
1,6.3,0.300,0.34,1.60,0.049,14.0,132.0,0.99400,3.30,0.49,9.500000,Normal
2,8.1,0.280,0.40,6.90,0.050,30.0,97.0,0.99510,3.26,0.44,10.100000,Normal
3,7.2,0.230,0.32,8.50,0.058,47.0,186.0,0.99560,3.19,0.40,9.900000,Normal
4,7.2,0.230,0.32,8.50,0.058,47.0,186.0,0.99560,3.19,0.40,9.900000,Normal
5,8.1,0.280,0.40,6.90,0.050,30.0,97.0,0.99510,3.26,0.44,10.100000,Normal
6,6.2,0.320,0.16,7.00,0.045,30.0,136.0,0.99490,3.18,0.47,9.600000,Normal
7,7.0,0.270,0.36,20.70,0.045,45.0,170.0,1.00100,3.00,0.45,8.800000,Normal
8,6.3,0.300,0.34,1.60,0.049,14.0,132.0,0.99400,3.30,0.49,9.500000,Normal
9,8.1,0.220,0.43,1.50,0.044,28.0,129.0,0.99380,3.22,0.45,11.000000,Normal


In [37]:
wq_X= StandardScaler().fit_transform(wq_X)

In [38]:
out = './ICA/'
kurt = {}
for dim in dims:
    ica.set_params(n_components=dim)
    tmp = ica.fit_transform(wq_X)
    tmp = pd.DataFrame(tmp)
    tmp = tmp.kurt(axis=0)
    kurt[dim] = tmp.abs().mean()

kurt = pd.Series(kurt) 
kurt.to_csv(out+'wine_quality.csv')

In [39]:
out = './ICA/'
nn_arch= [(10,),(10,20),(20,30),(50,50),(50,),(25,),(25,25),(100,25,100)]
nn_reg = [10**-x for x in range(1,5)]
clusters =  [2,5,10,15,20,25,30,35,40]
dims = [2,5,7]
grid ={'ica__n_components':dims,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
ica = FastICA(random_state=5)       
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
pipe = Pipeline([('ica',ica),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10,cv=5)

gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'Wine_Quality_dim.csv')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(10,), score=0.2163265306122449, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(10,), score=0.2163265306122449, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(10,), score=0.2163265306122449, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(10,), score=0.21654749744637386, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(10,) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s


[CV]  NN__alpha=0.1, ica__n_components=2, NN__hidden_layer_sizes=(10,), score=0.21654749744637386, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=5, NN__hidden_layer_sizes=(10,), score=0.2163265306122449, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=5, NN__hidden_layer_sizes=(10,), score=0.2163265306122449, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=5, NN__hidden_layer_sizes=(10,), score=0.2163265306122449, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=5, NN__hidden_layer_sizes=(10,), score=0.21654749744637386, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=5, NN__hidden_layer_sizes=(10,) 


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.4s remaining:    0.0s


[CV]  NN__alpha=0.1, ica__n_components=5, NN__hidden_layer_sizes=(10,), score=0.21654749744637386, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=7, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=7, NN__hidden_layer_sizes=(10,), score=0.7836734693877551, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=7, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=7, NN__hidden_layer_sizes=(10,), score=0.7836734693877551, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=7, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=7, NN__hidden_layer_sizes=(10,), score=0.7826530612244897, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=7, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=7, NN__hidden_layer_sizes=(10,), score=0.7834525025536262, total=   0.0s
[CV] NN__alpha=0.1, ica__n_components=7, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, ica__n_components=7, NN__hidden_layer_sizes=(10,), score=0.7834

...

[CV]  NN__alpha=0.001, ica__n_components=5, NN__hidden_layer_sizes=(25,), score=0.7836734693877551, total=   0.0s
[CV] NN__alpha=0.001, ica__n_components=5, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, ica__n_components=5, NN__hidden_layer_sizes=(25,), score=0.7834525025536262, total=   0.0s
[CV] NN__alpha=0.001, ica__n_components=5, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, ica__n_components=5, NN__hidden_layer_sizes=(25,), score=0.7834525025536262, total=   0.0s
[CV] NN__alpha=0.001, ica__n_components=7, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, ica__n_components=7, NN__hidden_layer_sizes=(25,), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.001, ica__n_components=7, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, ica__n_components=7, NN__hidden_layer_sizes=(25,), score=0.7520408163265306, total=   0.1s
[CV] NN__alpha=0.001, ica__n_components=7, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, ica__n_components=7, NN__hidden_layer_size

[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(10,), score=0.2163265306122449, total=   0.0s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(10,), score=0.2163265306122449, total=   0.0s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(10,), score=0.2163265306122449, total=   0.0s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(10,), score=0.21654749744637386, total=   0.0s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(10,), score=0.21654749744637386, total=   0.0s
[CV] NN__alpha=0.0001, ica__n_components=7, NN__hidde

[CV]  NN__alpha=0.0001, ica__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(50, 50), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(50, 50), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(50, 50), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(50, 50), score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, i

[CV]  NN__alpha=0.0001, ica__n_components=2, NN__hidden_layer_sizes=(25, 25), score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(25, 25), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(25, 25), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(25, 25), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(25, 25), score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, ica__n_components=5, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, i

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:   36.7s finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

### PCA on Wine Quality

In [40]:
from sklearn.decomposition import PCA

out = './PCA/'
cmap = cm.get_cmap('Spectral')

In [46]:
pca = PCA(random_state=5)
pca.fit(wq_X)
tmp = pd.Series(data = pca.explained_variance_,index = range(1,12))
tmp.to_csv(out+'Wine_Quality.csv')

In [47]:
grid ={'pca__n_components':dims,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
pca = PCA(random_state=5)       
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
pipe = Pipeline([('pca',pca),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10,cv=5)

gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'Wine_quality_dim.csv')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] NN__alpha=0.1, pca__n_components=2, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, pca__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7806122448979592, total=   0.1s
[CV] NN__alpha=0.1, pca__n_components=2, NN__hidden_layer_sizes=(10,) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV]  NN__alpha=0.1, pca__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7765306122448979, total=   0.1s
[CV] NN__alpha=0.1, pca__n_components=2, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, pca__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7826530612244897, total=   0.1s
[CV] NN__alpha=0.1, pca__n_components=2, NN__hidden_layer_sizes=(10,) 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s


[CV]  NN__alpha=0.1, pca__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.1, pca__n_components=2, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.1, pca__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7752808988764045, total=   0.1s
[CV] NN__alpha=0.1, pca__n_components=5, NN__hidden_layer_sizes=(10,) 


...

[CV]  NN__alpha=0.001, pca__n_components=2, NN__hidden_layer_sizes=(25,), score=0.7650663942798774, total=   0.0s
[CV] NN__alpha=0.001, pca__n_components=5, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, pca__n_components=5, NN__hidden_layer_sizes=(25,), score=0.7877551020408163, total=   0.1s
[CV] NN__alpha=0.001, pca__n_components=5, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, pca__n_components=5, NN__hidden_layer_sizes=(25,), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.001, pca__n_components=5, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, pca__n_components=5, NN__hidden_layer_sizes=(25,), score=0.8081632653061225, total=   0.1s
[CV] NN__alpha=0.001, pca__n_components=5, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, pca__n_components=5, NN__hidden_layer_sizes=(25,), score=0.797752808988764, total=   0.1s
[CV] NN__alpha=0.001, pca__n_components=5, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, pca__n_components=5, NN__hidden_layer_sizes

[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7826530612244897, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7763023493360572, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, pca__n_components=5, NN__hidden_layer_sizes=(10,), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, pca__n_components=5, NN__hidden_layer_sizes=(10,), score=0.7826530612244897, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, pca__n_components=5, NN__hidden

[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.7814096016343207, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.7722165474974464, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=5, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, pca__n_components=5, NN__hidden_layer_sizes=(50, 50), score=0.8112244897959183, total=   0.2s
[CV] NN__alpha=0.0001, pca__n_components=5, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, p

[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(25, 25), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(25, 25), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(25, 25), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(25, 25), score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, pca__n_components=2, NN__hidden_layer_sizes=(25, 25), score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, pca__n_components=5, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, p

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:   59.3s finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

## Random Projection

In [51]:
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score as acc
from sklearn.mixture import GaussianMixture as GMM
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_selection import mutual_info_classif as MIC
from sklearn.base import TransformerMixin,BaseEstimator
import scipy.sparse as sps
from scipy.linalg import pinv
from collections import defaultdict

def cluster_acc(Y,clusterLabels):
    assert (Y.shape == clusterLabels.shape)
    pred = np.empty_like(Y)
    for label in set(clusterLabels):
        mask = clusterLabels == label
        sub = Y[mask]
        target = Counter(sub).most_common(1)[0][0]
        pred[mask] = target    
    return acc(Y,pred)


class myGMM(GMM):
    def transform(self,X):
        return self.predict_proba(X)
        
        
def pairwiseDistCorr(X1,X2):
    assert X1.shape[0] == X2.shape[0]
    
    d1 = pairwise_distances(X1)
    d2 = pairwise_distances(X2)
    return np.corrcoef(d1.ravel(),d2.ravel())[0,1]

    
def aveMI(X,Y):    
    MI = MIC(X,Y) 
    return np.nanmean(MI)
    
  
def reconstructionError(projections,X):
    W = projections.components_
    if sps.issparse(W):
        W = W.todense()
    p = pinv(W)
    reconstructed = ((p@W)@(X.T)).T
    errors = np.square(X-reconstructed)
    return np.nanmean(errors)
    
    
        
#       
class ImportanceSelect(BaseEstimator, TransformerMixin):
    def __init__(self, model, n=1):
         self.model = model
         self.n = n
    def fit(self, *args, **kwargs):
         self.model.fit(*args, **kwargs)
         return self
    def transform(self, X):
         return X[:,self.model.feature_importances_.argsort()[::-1][:self.n]]
                  

from scipy.spatial import distance     
def compute_bic(kmeans,X):
    """
    Computes the BIC metric for a given clusters

    Parameters:
    -----------------------------------------
    kmeans:  List of clustering object from scikit learn

    X     :  multidimension np array of data points

    Returns:
    -----------------------------------------
    BIC value
    """

    centers = [kmeans.cluster_centers_]
    labels  = kmeans.labels_

    m = kmeans.n_clusters
    n = np.bincount(labels)
    N, d = X.shape

    
    cl_var = (1.0 / (N - m) / d) * sum([sum(distance.cdist(X[np.where(labels == i)], [centers[0][i]], 'euclidean')**2) for i in range(m)])

    const_term = 0.5 * m * np.log(N) * (d+1)

    BIC = np.sum([n[i] * np.log(n[i]) -
               n[i] * np.log(N) -
             ((n[i] * d) / 2) * np.log(2*np.pi*cl_var) -
             ((n[i] - 1) * d/ 2) for i in range(m)]) - const_term

    return(BIC)                  

In [52]:
from sklearn.random_projection import SparseRandomProjection, GaussianRandomProjection
from itertools import product
from matplotlib import cm

out = './RP/'
cmap = cm.get_cmap('Spectral') 

tmp = defaultdict(dict)
for i,dim in product(range(14),dims):
    rp = SparseRandomProjection(random_state=i, n_components=dim)
    tmp[dim][i] = pairwiseDistCorr(rp.fit_transform(wq_X), wq_X)
tmp = pd.DataFrame(tmp).T
tmp.to_csv(out+'Wine Quality.csv')

In [53]:
grid ={'rp__n_components':dims,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
rp = SparseRandomProjection(random_state=5)       
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
pipe = Pipeline([('rp',rp),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10,cv=5)

gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'Wine Quality dim red.csv')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] NN__alpha=0.1, rp__n_components=2, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.1, rp__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7846938775510204, total=   0.1s
[CV] NN__alpha=0.1, rp__n_components=2, NN__hidden_layer_sizes=(10,) .


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV]  NN__alpha=0.1, rp__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7377551020408163, total=   0.1s
[CV] NN__alpha=0.1, rp__n_components=2, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.1, rp__n_components=2, NN__hidden_layer_sizes=(10,), score=0.786734693877551, total=   0.1s
[CV] NN__alpha=0.1, rp__n_components=2, NN__hidden_layer_sizes=(10,) .


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.6s remaining:    0.0s


[CV]  NN__alpha=0.1, rp__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7660878447395302, total=   0.1s
[CV] NN__alpha=0.1, rp__n_components=2, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.1, rp__n_components=2, NN__hidden_layer_sizes=(10,), score=0.7609805924412666, total=   0.1s
[CV] NN__alpha=0.1, rp__n_components=5, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.1, rp__n_components=5, NN__hidden_layer_sizes=(10,), score=0.786734693877551, total=   0.2s
[CV] NN__alpha=0.1, rp__n_components=5, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.1, rp__n_components=5, NN__hidden_layer_sizes=(10,), score=0.7857142857142857, total=   0.2s
[CV] NN__alpha=0.1, rp__n_components=5, NN__hidden_layer_sizes=(10,) .


...

[CV]  NN__alpha=0.001, rp__n_components=5, NN__hidden_layer_sizes=(25,), score=0.7826530612244897, total=   0.1s
[CV] NN__alpha=0.001, rp__n_components=5, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, rp__n_components=5, NN__hidden_layer_sizes=(25,), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.001, rp__n_components=5, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, rp__n_components=5, NN__hidden_layer_sizes=(25,), score=0.7824310520939735, total=   0.1s
[CV] NN__alpha=0.001, rp__n_components=5, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, rp__n_components=5, NN__hidden_layer_sizes=(25,), score=0.7783452502553626, total=   0.1s
[CV] NN__alpha=0.001, rp__n_components=7, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, rp__n_components=7, NN__hidden_layer_sizes=(25,), score=0.7918367346938775, total=   0.1s
[CV] NN__alpha=0.001, rp__n_components=7, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, rp__n_components=7, NN__hidden_layer_sizes=(25,), sc

[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(10,), score=0.7846938775510204, total=   0.1s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(10,), score=0.7857142857142857, total=   0.2s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(10,), score=0.8030612244897959, total=   0.1s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(10,), score=0.7119509703779366, total=   0.1s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(10,), score=0.7722165474974464, total=   0.2s
[CV] NN__alpha=0.0001, rp__n_components=7, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.0001, rp__n_components=7, NN__hidden_layer_size

[CV]  NN__alpha=0.0001, rp__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.7824310520939735, total=   0.1s
[CV] NN__alpha=0.0001, rp__n_components=2, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, rp__n_components=2, NN__hidden_layer_sizes=(50, 50), score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(50, 50), score=0.7969387755102041, total=   0.2s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(50, 50), score=0.7857142857142857, total=   0.1s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(50, 50), score=0.8377551020408164, total=   0.3s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(50, 50) 
[CV]  NN__alpha=0.0001, rp__n_compo

[CV]  NN__alpha=0.0001, rp__n_components=2, NN__hidden_layer_sizes=(25, 25), score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(25, 25), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(25, 25), score=0.7816326530612245, total=   0.1s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(25, 25), score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(25, 25), score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, rp__n_components=5, NN__hidden_layer_sizes=(25, 25) 
[CV]  NN__alpha=0.0001, rp__n_compo

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:   56.4s finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

### Random Forest

In [54]:
from sklearn.ensemble import RandomForestClassifier

In [55]:
rfc = RandomForestClassifier(n_estimators=100,class_weight='balanced',random_state=5,n_jobs=7)
fs_wq = rfc.fit(wq_X,wq_Y).feature_importances_ 
fs_bc = rfc.fit(bc_X,bc_Y).feature_importances_ 

In [56]:
out = './RF/'
tmp = pd.Series(np.sort(fs_wq)[::-1])
tmp.to_csv(out+'wine_quality.csv')
tmp = pd.Series(np.sort(fs_bc)[::-1])
tmp.to_csv(out+'breast_cancer.csv')

In [57]:
filtr = ImportanceSelect(rfc)
grid ={'filter__n':dims,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
pipe = Pipeline([('filter',filtr),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10,cv=5)
    
gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'Wine Quality.csv')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=2 ........
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=2, score=0.786734693877551, total=   0.6s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=2 ........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=2, score=0.7877551020408163, total=   0.6s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=2 ........


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.5s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=2, score=0.7816326530612245, total=   0.6s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=2 ........


...

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), filter__n=5, score=0.7857142857142857, total=   0.6s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), filter__n=5 ...
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), filter__n=5, score=0.789795918367347, total=   0.6s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), filter__n=5 ...
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), filter__n=5, score=0.789795918367347, total=   0.6s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), filter__n=5 ...
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), filter__n=5, score=0.8059244126659857, total=   0.6s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), filter__n=5 ...
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), filter__n=5, score=0.7824310520939735, total=   0.6s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), filter__n=7 ...
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), filter__n=7, score=0.7836734693877551, total= 

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10, 20), filter__n=5, score=0.7836734693877551, total=   0.6s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10, 20), filter__n=5 ..
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10, 20), filter__n=5, score=0.786734693877551, total=   0.7s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10, 20), filter__n=5 ..
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10, 20), filter__n=5, score=0.8214285714285714, total=   0.6s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10, 20), filter__n=5 ..
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10, 20), filter__n=5, score=0.7885597548518897, total=   0.6s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10, 20), filter__n=5 ..
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10, 20), filter__n=5, score=0.7875383043922369, total=   0.6s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10, 20), filter__n=7 ..
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10, 20), filter__n=7, score=0.7836734693877551, 

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5, score=0.7836734693877551, total=   0.6s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5 .....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5, score=0.7836734693877551, total=   0.6s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5 .....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5, score=0.7773237997957099, total=   0.6s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5 .....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5, score=0.7936670071501533, total=   0.6s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=7 .....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=7, score=0.8020408163265306, total=   0.7s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=7 .....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=7, score=0.7887755102040817, total=   0.7s
[CV

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=5, score=0.8193877551020409, total=   0.7s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=5 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=5, score=0.7906026557711952, total=   0.8s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=5 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=5, score=0.7824310520939735, total=   0.8s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7, score=0.8112244897959183, total=   0.7s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7, score=0.7938775510204081, total=   0.7s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7 
[CV]  NN__alpha=0.0001, NN__hidden_layer_size

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  6.1min finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

In [58]:
dim = 7
filtr = ImportanceSelect(rfc,dim)    
wq_X2 = filtr.fit_transform(wq_X,wq_Y)
wq_2 = pd.DataFrame(np.hstack((wq_X2,np.atleast_2d(wq_Y).T)))
cols = list(range(wq_2.shape[1]))
cols[-1] = 'Class'
wq_2.columns = cols
wq_2.to_csv(out+'wq datasets.csv')

In [59]:
dims = [5,10,15,20,25]
filtr = ImportanceSelect(rfc)

grid ={'filter__n':dims,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}  
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
pipe = Pipeline([('filter',filtr),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10,cv=5)
    




In [60]:
gs.fit(bc_X,bc_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'breast cancer dim red.csv')

Fitting 5 folds for each of 160 candidates, totalling 800 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=5 ........
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=5, score=0.7043478260869566, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=5 ........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=5, score=0.6956521739130435, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=5 ........


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=5, score=0.8053097345132744, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), filter__n=5 ........


...

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), filter__n=5, score=0.9557522123893806, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), filter__n=10 .
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), filter__n=10, score=0.8695652173913043, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), filter__n=10 .
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), filter__n=10, score=0.9217391304347826, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), filter__n=10 .
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), filter__n=10, score=0.9203539823008849, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), filter__n=10 .
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), filter__n=10, score=0.9292035398230089, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), filter__n=10 .
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), filter__n=10, score=0.893805309734

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5, score=0.8260869565217391, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5 .....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5, score=0.8956521739130435, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5 .....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5, score=0.9292035398230089, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5 .....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5, score=0.9292035398230089, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5 .....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=5, score=0.9203539823008849, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=10 ....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50,), filter__n=10, score=0.9043478260869565, total=   0.4s
[C

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), filter__n=25, score=0.9304347826086956, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), filter__n=25 ....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), filter__n=25, score=0.8672566371681416, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), filter__n=25 ....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), filter__n=25, score=0.8938053097345132, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), filter__n=25 ....
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), filter__n=25, score=0.8849557522123894, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25, 25), filter__n=5 ..
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25, 25), filter__n=5, score=0.9043478260869565, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25, 25), filter__n=5 ..
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25, 25), filter__n=5, score=0.9304347826086956, total= 

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=20, score=0.9217391304347826, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=20 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=20, score=0.9292035398230089, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=20 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=20, score=0.8938053097345132, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=20 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=20, score=0.9557522123893806, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=25 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=25, score=0.9130434782608695, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), filter__n=25 
[CV]  NN__alpha=0.0001, NN__hidden_

[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  7.6min finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

In [62]:
dim = 15
filtr = ImportanceSelect(rfc,dim)
bc_X2 = filtr.fit_transform(bc_X,bc_Y)
bc_2 = pd.DataFrame(np.hstack((bc_X2,np.atleast_2d(bc_Y).T)))
cols = list(range(bc_2.shape[1]))
cols[-1] = 'Class'
bc_2.columns = cols
bc_2.to_csv(out+'bc_datasets.csv')

###  PART 5 - Treating clustering algorithms as dimentionality reduction algorithms.

In [64]:
clusters =  [2,5,10]
from sklearn.cluster import KMeans as kmeans
out = './PCA/'
grid ={'km__n_clusters':clusters,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
km = kmeans(random_state=5)
pca = PCA(random_state=5, n_components=5)
pipe = Pipeline([('pca',pca),('km',km),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10)

gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'wine quality pca Kmeans.csv')

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,) ...
[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21664626682986537, total=   0.1s
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,) ...
[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.1s
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,) ...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s


[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.1s
[CV] NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,) ...
[CV]  NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7833537331701347, total=   0.2s
[CV] NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,) ...
[CV]  NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,) ...


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.6s remaining:    0.0s


[CV]  NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.1, km__n_clusters=10, NN__hidden_layer_sizes=(10,) ..


...

[CV]  NN__alpha=0.001, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.1s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7833537331701347, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(10,), score=0.7760097919216646, total=   0.3s
[CV] NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(10,), score=0.80453431

[CV]  NN__alpha=0.001, km__n_clusters=2, NN__hidden_layer_sizes=(25,), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), score=0.7833537331701347, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(25,), score=0.7827417380660955, total=   0.3s
[CV] NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(25,), score=0.783700980

[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(20, 30), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(20, 30), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30), score=0.7833537331701347, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=10, NN__hidden_

[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100), score=0.7833537331701347, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(100, 25, 

[Parallel(n_jobs=1)]: Done 288 out of 288 | elapsed:  1.0min finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

In [65]:
from sklearn.mixture import GaussianMixture as GMM
out = './PCA/'
grid ={'gmm__n_components':clusters,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
gmm = myGMM(random_state=5)
pipe = Pipeline([('pca',pca),('gmm',gmm),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10,cv=5)

gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'wine quality pca EM.csv')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.5s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=5 
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=5, score=0.2163265306122449, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=5 


...

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(50,), gmm__n_components=10, score=0.7834525025536262, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_comp

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(100, 25, 100), gmm__n_components=10, score=0.8214285714285714, total=   0.5s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(100, 25, 100), gmm__n_components=10 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(100, 25, 100), gmm__n_components=10, score=0.7834525025536262, total=   0.4s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(100, 25, 100), gmm__n_components=10 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(100, 25, 100), gmm__n_components=10, score=0.7834525025536262, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7836734693877551, total=   0.2s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7834525025536262, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7834525025536262, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50, 50), gmm__n_components=2 
[CV]  NN__alpha=

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=5, score=0.7834525025536262, total=   0.2s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=5 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=5, score=0.7834525025536262, total=   0.2s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  1.7min finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

In [67]:
clusters =  [2,5,10]

out = './ICA/'
grid ={'km__n_clusters':clusters,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
km = kmeans(random_state=5)
ica = FastICA(random_state=5, max_iter=1000, n_components=5)  
pipe = Pipeline([('ica',ica),('km',km),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10)

gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'wine quality ica Kmeans.csv')

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,) ...
[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21664626682986537, total=   0.1s
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,) ...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.1s
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,) ...
[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.1s
[CV] NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,) ...


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s


[CV]  NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.21664626682986537, total=   0.2s
[CV] NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,) ...
[CV]  NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.2s
[CV] NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,) ...


...

[CV]  NN__alpha=0.001, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.1s
[CV] NN__alpha=0.001, km__n_clusters=2, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.001, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.1s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.21664626682986537, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(10,), score=0.78335

[CV]  NN__alpha=0.001, km__n_clusters=2, NN__hidden_layer_sizes=(25,), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.001, km__n_clusters=2, NN__hidden_layer_sizes=(25,) .
[CV]  NN__alpha=0.001, km__n_clusters=2, NN__hidden_layer_sizes=(25,), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), score=0.7833537331701347, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(25,), score=0.7833537331

[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(20, 30), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(20, 30), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30), score=0.7833537331701347, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=10, NN__hidden_

[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100), score=0.7833537331701347, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(100, 25, 

[Parallel(n_jobs=1)]: Done 288 out of 288 | elapsed:  1.0min finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

In [68]:
grid ={'gmm__n_components':clusters,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
gmm = myGMM(random_state=5)
pipe = Pipeline([('ica',ica),('gmm',gmm),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10,cv=5)

gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'wine quality ica EM.csv')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.5s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=5 
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=5, score=0.2163265306122449, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=5 


...

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(50,), gmm__n_components=10, score=0.7834525025536262, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_comp

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(100, 25, 100), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(100, 25, 100), gmm__n_components=10 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(100, 25, 100), gmm__n_components=10, score=0.8008171603677222, total=   0.4s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(100, 25, 100), gmm__n_components=10 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(100, 25, 100), gmm__n_components=10, score=0.7834525025536262, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7834525025536262, total=   0.2s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7834525025536262, total=   0.2s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50, 50), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50, 50), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50, 50), gmm__n_components=2 
[CV]  NN__alpha=0.

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=5, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7836734693877551, total=   0.2s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7836734693877551, total=   0.2s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7609805924412666, total=   0.2s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  1.4min finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

In [69]:
out = './RP/'
grid ={'km__n_clusters':clusters,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
km = kmeans(random_state=5)
rp = SparseRandomProjection(random_state=5, n_components=7)  
pipe = Pipeline([('rp',rp),('km',km),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10)

gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'wine quality rp Kmeans.csv')

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,) ...
[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21664626682986537, total=   0.1s
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,) ...
[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.1s
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,) ...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s


[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.1s
[CV] NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,) ...
[CV]  NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7784577723378213, total=   0.2s
[CV] NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,) ...


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.6s remaining:    0.0s


[CV]  NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7634803921568627, total=   0.1s
[CV] NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,) ...
[CV]  NN__alpha=0.1, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7787990196078431, total=   0.2s
[CV] NN__alpha=0.1, km__n_clusters=10, NN__hidden_layer_sizes=(10,) ..


...

[CV]  NN__alpha=0.001, km__n_clusters=2, NN__hidden_layer_sizes=(10,), score=0.21629901960784315, total=   0.1s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7784577723378213, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7634803921568627, total=   0.1s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), score=0.7787990196078431, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(10,), score=0.7674418604651163, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(10,) 
[CV]  NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(10,), score=0.76593137

[CV]  NN__alpha=0.001, km__n_clusters=2, NN__hidden_layer_sizes=(25,), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), score=0.7833537331701347, total=   0.1s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,) .
[CV]  NN__alpha=0.001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(25,), score=0.7833537331701347, total=   0.3s
[CV] NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(25,) 
[CV]  NN__alpha=0.001, km__n_clusters=10, NN__hidden_layer_sizes=(25,), score=0.783700980

[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(20, 30), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(20, 30), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30), score=0.7833537331701347, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30), score=0.7800245098039216, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(20, 30), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(20, 30) 
[CV]  NN__alpha=0.0001, km__n_clusters=10, NN__hidden_

[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.1s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100), score=0.7833537331701347, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100) 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(100, 25, 100), score=0.7837009803921569, total=   0.2s
[CV] NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(100, 25, 

[Parallel(n_jobs=1)]: Done 288 out of 288 | elapsed:   55.9s finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

In [70]:
grid ={'gmm__n_components':clusters,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
gmm = myGMM(random_state=5)
pipe = Pipeline([('rp',rp),('gmm',gmm),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10,cv=5)

gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'wine quality rp EM.csv')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(10,), gmm__n_components=5 


...

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=2, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=5 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=5, score=0.7836734693877551, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=5 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=5, score=0.7836734693877551, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_components=5 
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), gmm__n_compo

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(100, 25, 100), gmm__n_components=10, score=0.7834525025536262, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2, score=0.7834525025536262, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(10,), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7834525025536262, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(20, 30), gmm__n_components=10, score=0.7834525025536262, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50, 50), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50, 50), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50, 50), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(50, 50), gmm__n_components=2, score=0.7836734693877551, total=   0.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(50, 50), gmm__n_components=2 
[CV]  NN__alpha=0.00

[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7836734693877551, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7775510204081633, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7834525025536262, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10 
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(25,), gmm__n_components=10, score=0.7834525025536262, total=   0.3s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(25, 25), gmm__n_components=2 
[CV]  NN__alpha=0.0001, NN__hidden_layer_si

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  1.7min finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

In [72]:
out = "./RF/"
filtr = ImportanceSelect(rfc)
grid ={'filter__n':[7],'km__n_clusters':clusters,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
km = kmeans(random_state=5)
pipe = Pipeline([('filter',filtr),('km',km),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10,cv=5)

gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'Wine Quality RF Kmeans.csv')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), filter__n=7 
[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), filter__n=7, score=0.7836734693877551, total=   0.7s
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), filter__n=7 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), filter__n=7, score=0.2163265306122449, total=   0.6s
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), filter__n=7 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.5s remaining:    0.0s


[CV]  NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), filter__n=7, score=0.7836734693877551, total=   0.7s
[CV] NN__alpha=0.1, km__n_clusters=2, NN__hidden_layer_sizes=(10,), filter__n=7 


...

[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(10,), filter__n=7, score=0.7834525025536262, total=   0.6s
[CV] NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(10,), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(10,), filter__n=7, score=0.7640449438202247, total=   0.6s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), filter__n=7, score=0.7714285714285715, total=   0.8s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), filter__n=7, score=0.7836734693877551, total=   0.7s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(10,), filter__n=7, score=0.7928571428571428, total=   0.7s
[CV] NN__alpha=0.0001, km__n_clusters=5

[CV]  NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(20, 30), filter__n=7, score=0.8081632653061225, total=   0.9s
[CV] NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(20, 30), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(20, 30), filter__n=7, score=0.8038815117466803, total=   0.9s
[CV] NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(20, 30), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(20, 30), filter__n=7, score=0.7834525025536262, total=   0.8s
[CV] NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(50, 50), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(50, 50), filter__n=7, score=0.7836734693877551, total=   0.6s
[CV] NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(50, 50), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=2, NN__hidden_layer_sizes=(50, 50), filter__n=7, score=0.7836734693877551, total=   0.6s
[CV] NN

[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), filter__n=7, score=0.7836734693877551, total=   0.7s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), filter__n=7, score=0.7836734693877551, total=   0.7s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), filter__n=7, score=0.7834525025536262, total=   0.7s
[CV] NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=5, NN__hidden_layer_sizes=(25,), filter__n=7, score=0.7834525025536262, total=   0.7s
[CV] NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(25,), filter__n=7 
[CV]  NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(25,), filter__n=7, score=0.7836734693877551, total=   0.8s
[CV] NN__alpha=0.0001, km__n_clusters

[CV]  NN__alpha=0.0001, km__n_clusters=10, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7, score=0.7558733401430031, total=   1.0s


[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  7.1min finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn

In [75]:
out = "./RF/"
filtr = ImportanceSelect(rfc)
grid ={'filter__n':[7],'gmm__n_components':clusters,'NN__alpha':nn_reg,'NN__hidden_layer_sizes':nn_arch}
mlp = MLPClassifier(activation='relu',max_iter=2000,early_stopping=True,random_state=5)
gmm = myGMM(random_state=5)
pipe = Pipeline([('filter',filtr),('gmm',gmm),('NN',mlp)])
gs = GridSearchCV(pipe,grid,verbose=10,cv=5)

gs.fit(wq_X,wq_Y)
tmp = pd.DataFrame(gs.cv_results_)
tmp.to_csv(out+'Wine Quality RF EM.csv')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] NN__alpha=0.1, gmm__n_components=2, NN__hidden_layer_sizes=(10,), filter__n=7 
[CV]  NN__alpha=0.1, gmm__n_components=2, NN__hidden_layer_sizes=(10,), filter__n=7, score=0.7836734693877551, total=   0.6s
[CV] NN__alpha=0.1, gmm__n_components=2, NN__hidden_layer_sizes=(10,), filter__n=7 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV]  NN__alpha=0.1, gmm__n_components=2, NN__hidden_layer_sizes=(10,), filter__n=7, score=0.7836734693877551, total=   0.6s
[CV] NN__alpha=0.1, gmm__n_components=2, NN__hidden_layer_sizes=(10,), filter__n=7 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.5s remaining:    0.0s


[CV]  NN__alpha=0.1, gmm__n_components=2, NN__hidden_layer_sizes=(10,), filter__n=7, score=0.7836734693877551, total=   0.6s
[CV] NN__alpha=0.1, gmm__n_components=2, NN__hidden_layer_sizes=(10,), filter__n=7 


...

[CV]  NN__alpha=0.001, gmm__n_components=5, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7, score=0.7834525025536262, total=   0.8s
[CV] NN__alpha=0.001, gmm__n_components=10, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7 
[CV]  NN__alpha=0.001, gmm__n_components=10, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7, score=0.7836734693877551, total=   1.0s
[CV] NN__alpha=0.001, gmm__n_components=10, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7 
[CV]  NN__alpha=0.001, gmm__n_components=10, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7, score=0.7918367346938775, total=   0.9s
[CV] NN__alpha=0.001, gmm__n_components=10, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7 
[CV]  NN__alpha=0.001, gmm__n_components=10, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7, score=0.8244897959183674, total=   1.0s
[CV] NN__alpha=0.001, gmm__n_components=10, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7 
[CV]  NN__alpha=0.001, gmm__n_components=10, NN__hidden_layer_sizes=(

[CV]  NN__alpha=0.0001, gmm__n_components=2, NN__hidden_layer_sizes=(20, 30), filter__n=7, score=0.7836734693877551, total=   0.6s
[CV] NN__alpha=0.0001, gmm__n_components=2, NN__hidden_layer_sizes=(20, 30), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=2, NN__hidden_layer_sizes=(20, 30), filter__n=7, score=0.7834525025536262, total=   0.6s
[CV] NN__alpha=0.0001, gmm__n_components=2, NN__hidden_layer_sizes=(20, 30), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=2, NN__hidden_layer_sizes=(20, 30), filter__n=7, score=0.7834525025536262, total=   0.7s
[CV] NN__alpha=0.0001, gmm__n_components=5, NN__hidden_layer_sizes=(20, 30), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=5, NN__hidden_layer_sizes=(20, 30), filter__n=7, score=0.7836734693877551, total=   0.8s
[CV] NN__alpha=0.0001, gmm__n_components=5, NN__hidden_layer_sizes=(20, 30), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=5, NN__hidden_layer_sizes=(20, 30), filter__n=7, score=0.7836734693877551,

[CV]  NN__alpha=0.0001, gmm__n_components=10, NN__hidden_layer_sizes=(50,), filter__n=7, score=0.7836734693877551, total=   0.9s
[CV] NN__alpha=0.0001, gmm__n_components=10, NN__hidden_layer_sizes=(50,), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=10, NN__hidden_layer_sizes=(50,), filter__n=7, score=0.7887755102040817, total=   0.8s
[CV] NN__alpha=0.0001, gmm__n_components=10, NN__hidden_layer_sizes=(50,), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=10, NN__hidden_layer_sizes=(50,), filter__n=7, score=0.8122448979591836, total=   0.9s
[CV] NN__alpha=0.0001, gmm__n_components=10, NN__hidden_layer_sizes=(50,), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=10, NN__hidden_layer_sizes=(50,), filter__n=7, score=0.7824310520939735, total=   0.8s
[CV] NN__alpha=0.0001, gmm__n_components=10, NN__hidden_layer_sizes=(50,), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=10, NN__hidden_layer_sizes=(50,), filter__n=7, score=0.7834525025536262, total=   0.8s
[CV

[CV]  NN__alpha=0.0001, gmm__n_components=2, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7, score=0.7834525025536262, total=   0.7s
[CV] NN__alpha=0.0001, gmm__n_components=2, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=2, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7, score=0.7834525025536262, total=   0.7s
[CV] NN__alpha=0.0001, gmm__n_components=5, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=5, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7, score=0.7836734693877551, total=   0.8s
[CV] NN__alpha=0.0001, gmm__n_components=5, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=5, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7, score=0.7836734693877551, total=   0.8s
[CV] NN__alpha=0.0001, gmm__n_components=5, NN__hidden_layer_sizes=(100, 25, 100), filter__n=7 
[CV]  NN__alpha=0.0001, gmm__n_components=5, NN__hidden_layer_sizes=

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  6.9min finished
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nandan/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warn